In [97]:
import http.client
import config
import pandas as pd
import json
import folium

In [ ]:
def get_arrivals(airlines, dates):
    conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")
    conn2 = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': f"{config.api_key}",
        'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
    }

    df_arrivals = pd.DataFrame()  # Initialize an empty DataFrame
    df_departures = pd.DataFrame()  # Initialize an empty DataFrame

    for airline in airlines:
        for date in dates:
            conn.request("GET", f"/v2/airport/arrivals?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)
            conn2.request("GET", f"/v2/airport/departures?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)

            arrivals = conn.getresponse().read().decode('utf-8')
            departures = conn2.getresponse().read().decode('utf-8')

            arrivals = json.loads(arrivals)
            departures = json.loads(departures)

            arrivals_data = arrivals['result']['flights']
            departures_data = departures['result']['flights']

            df_arrivals_temp = pd.json_normalize(arrivals_data)  # Temporary DataFrame for each request
            df_departures_temp = pd.json_normalize(departures_data)  # Temporary DataFrame for each request

            df_arrivals = pd.concat([df_arrivals, df_arrivals_temp])  # Concatenate the temporary DataFrame with the main one

    return df_arrivals, df_departures

In [108]:
conn = http.client.HTTPSConnection("app.goflightlabs.com")
conn2 = http.client.HTTPSConnection("app.goflightlabs.com")
conn3 = http.client.HTTPSConnection("app.goflightlabs.com")
conn4 = http.client.HTTPSConnection("app.goflightlabs.com")

conn.request("GET", f"/flights?access_key={config.api_key}&depIata=ATL")
conn2.request("GET", f"/flights?access_key={config.api_key}&arrIata=ATL")
conn3.request("GET", f"/historical/2023-05-10?access_key={config.api_key}&code=ATL&type=departure")
conn4.request("GET", f"/historical/2023-05-10?access_key={config.api_key}&code=ATL&type=arrival")

real_time_departures = conn.getresponse().read().decode('utf-8')
real_time_departures = json.loads(real_time_departures)

real_time_arrivals = conn2.getresponse().read().decode('utf-8')
real_time_arrivals = json.loads(real_time_arrivals)

historical_departures = conn3.getresponse().read().decode('utf-8')
historical_departures = json.loads(historical_departures)

historical_arrivals = conn4.getresponse().read().decode('utf-8')
historical_arrivals = json.loads(historical_arrivals)

In [109]:
df_real_time_departures = pd.json_normalize(real_time_departures['data'])
df_real_time_arrials = pd.json_normalize(real_time_arrivals['data'])
df_real_time_flights = pd.concat([df_real_time_departures, df_real_time_arrials])
df_hist_departures = pd.json_normalize(historical_departures['data'])
df_hist_arrivals = pd.json_normalize(historical_arrivals['data'])
df_hist_flights = pd.concat([df_hist_departures, df_hist_arrivals])

In [110]:
df_hist_flights.info()

In [115]:
# df_departures.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Arrival_Airport_IATA',
#                               'departure.airport.city':'Arrival_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_departures = df_departures[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Arrival_Airport_IATA', 'Arrival_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]

In [116]:
# df_arrivals.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Departure_Airport_IATA',
#                               'departure.airport.city':'Departure_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_arrivals = df_arrivals[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Departure_Airport_IATA', 'Departure_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]